In [1]:
import os

In [2]:
%pwd

'c:\\Users\\gupta\\Documents\\GitHub\\Brain_Tumor_Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\gupta\\Documents\\GitHub\\Brain_Tumor_Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_optimizer: str
    params_loss: str
    params_metrics: list
    params_freeze_layers: int

In [6]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.input_shape,
            params_learning_rate=self.params.learning_rate,
            params_include_top=self.params.include_top,
            params_weights=self.params.weights,
            params_classes=self.params.classes,
            params_optimizer= self.params.optimizer,
            params_loss= self.params.loss,
            params_metrics= self.params.metrics,
            params_freeze_layers= self.params.freeze_layers,
        )

        return prepare_base_model_config


In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [13]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    
    def get_base_model(self):
        # Step 1: Load the ResNet50 model with pre-trained weights, adjusting for grayscale input (1 channel)
        input_tensor = tf.keras.layers.Input(shape=self.config.params_image_size)  # (224, 224, 1) for grayscale input
        
        # Load the ResNet50 model without weights for the initial build (weights=None)
        self.model = tf.keras.applications.ResNet50(
            input_tensor=input_tensor,
            weights=None,  # Don't load pre-trained weights yet
            include_top=self.config.params_include_top
        )

        # Step 2: Load pre-trained ResNet50 model with ImageNet weights (3 channels)
        pre_trained_model = tf.keras.applications.ResNet50(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)  # RGB input shape
        )

         # Step 3: Copy weights from the pre-trained model, except for the first layer
        for layer in self.model.layers:
            if layer.name != 'conv1_conv':  # Skip the first conv layer, which has different input shape
                try:
                    layer.set_weights(pre_trained_model.get_layer(layer.name).get_weights())
                except:
                    # Some layers may not have weights (like pooling or dropout), so we skip them
                    pass


        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_layers, learning_rate,optimizer_name,loss_function, metrics):
         # Fine-tuning: freeze layers except the first few
        for layer in model.layers[:freeze_layers]:
            layer.trainable = True  # Unfreeze the first few layers

        for layer in model.layers[freeze_layers:]:
            layer.trainable = False  # Freeze the remaining layers

        # Add global pooling and dense layer for classification
        x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax" if classes > 2 else "sigmoid"  # Softmax for multi-class, sigmoid for binary
        )(x)

         # Dynamically get the optimizer class from tf.keras.optimizers
        optimizer_class = getattr(tf.keras.optimizers, optimizer_name)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=optimizer_class(learning_rate=learning_rate),
            loss=loss_function, 
            metrics=metrics
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_layers=self.config.params_freeze_layers,  # Unfreeze the first few layers
            learning_rate=self.config.params_learning_rate,
            optimizer_name=self.config.params_optimizer,  # Use optimizer name from config
            loss_function=self.config.params_loss,  # Use loss function from config
            metrics=self.config.params_metrics  # Use metrics from config
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    


In [14]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-09-04 16:06:39,467: INFO: common: yaml file: config\config.yaml loaded successfully]


[2024-09-04 16:06:39,493: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-04 16:06:39,496: INFO: common: created directory at: artifacts]
[2024-09-04 16:06:39,499: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-09-04 16:06:45,512: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 224, 224, 1)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 1)          0         ['input_4[0][0]']             
                                        

Explanation of the Output:

Model Summary:

Your model has been initialized with the grayscale input shape (224, 224, 1), as shown by the InputLayer entry (input_4 (InputLayer)).
The first layer (conv1_conv (Conv2D)) has been successfully adapted to work with grayscale input (1 channel). The output shape of this layer is (112, 112, 64), which matches the expected output for ResNet-50's architecture.
The total number of parameters (Total params: 23,585,538) and the breakdown into trainable and non-trainable parameters seem appropriate for a model like ResNet-50.

Warnings:

The warning Compiled the loaded model, but the compiled metrics have yet to be built is a standard message. It indicates that while the model has been compiled, you haven't yet called .fit() or .evaluate() on the model, so the metrics have not been initialized. This will resolve itself once you start training or evaluating the model.

Trainable vs Non-Trainable Parameters:

It looks like the majority of the model's parameters are non-trainable (23573824 non-trainable params), which is expected since most of the layers were frozen. Only a small portion (11714 trainable params) is trainable, likely from the layers that you have unfrozen for fine-tuning.
This is good because you’re leveraging the pre-trained weights from ImageNet, while fine-tuning only specific layers.